In [14]:
import copy
import os
from os.path import join as oj
import glob
import argparse
import pickle as pkl
import time
import warnings
from scipy import stats
import dask
from dask.distributed import Client
import numpy as np
import pandas as pd
from tqdm import tqdm
import sys
from collections import defaultdict
from typing import Callable, List, Tuple
import itertools
from sklearn.metrics import roc_auc_score, f1_score, recall_score, precision_score, mean_squared_error

sys.path.append(".")
sys.path.append("..")
sys.path.append("../../imodels/")

warnings.filterwarnings("ignore", message="Bins whose width")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, mean_absolute_error, accuracy_score, roc_auc_score, mean_squared_error

from imodels.importance import RandomForestPlusRegressor, RandomForestPlusClassifier, \
    RidgeRegressorPPM, LassoRegressorPPM, IdentityTransformer
from imodels.importance.rf_plus import _fast_r2_score
import seaborn as sns
from util import ModelConfig, FIModelConfig, tp, fp, neg, pos, specificity_score, auroc_score, auprc_score, compute_nsg_feat_corr_w_sig_subspace, apply_splitting_strategy
import shap
import sklearn

In [15]:
def sample_normal_X_subgroups(n, d, mean, scale):
    """
    :param n: Number of samples
    :param d: Number of features
    :param mean: Nested list of mean of normal distribution for each subgroup
    :param scale: Nested ist of scale of normal distribution for each subgroup
    :return:
    """
    np.random.seed(0)
    assert len(mean[0]) == len(scale[0]) == d
    num_groups = len(mean)
    result = []
    group_size = n // num_groups
    for i in range(num_groups):
        X = np.zeros((group_size, d))
        for j in range(d):
            X[:, j] = np.random.normal(mean[i][j], scale[i][j], size=group_size)
        result.append(X)
    return np.vstack(result)

In [16]:
def generate_random_shuffle(data, seed):
    """
    Randomly shuffle each column of the data.
    """
    np.random.seed(seed)
    return np.array([np.random.permutation(data[:, i]) for i in range(data.shape[1])]).T


def ablation(data, feature_importance, mode, num_features, seed):
    """
    Replace the top num_features max feature importance data with random shuffle for each sample
    """
    assert mode in ["max", "min"]
    fi = feature_importance.to_numpy()
    shuffle = generate_random_shuffle(data.copy(), seed)
    if mode == "max":
        indices = np.argsort(-fi)
    else:
        indices = np.argsort(fi)
    data_copy = data.copy()
    for i in range(data.shape[0]):
        for j in range(num_features):
            data_copy[i, indices[i,j]] = shuffle[i, indices[i,j]]
    return data_copy


In [17]:
def tree_shap_local(X, y, fit):
    """
    Compute average treeshap value across observations.
    Larger absolute values indicate more important features.
    :param X: design matrix
    :param y: response
    :param fit: fitted model of interest (tree-based)
    :return: dataframe of shape: (n_samples, n_features)
    """
    explainer = shap.TreeExplainer(fit)
    shap_values = explainer.shap_values(X, check_additivity=False)
    if sklearn.base.is_classifier(fit):
        # Shape values are returned as a list of arrays, one for each class
        def add_abs(a, b):
            return abs(a) + abs(b)
        results = reduce(add_abs, shap_values)
    else:
        results = abs(shap_values)
    result_table = pd.DataFrame(results, columns=[f'Feature_{i}' for i in range(X.shape[1])])

    return result_table

In [18]:
# import sys
# sys.path.append(".")
# sys.path.append("..")

In [19]:
# def sample_real_X(fpath=None, X=None, seed=None, normalize=True,
#                   sample_row_n=None, sample_col_n=None, permute_col=True,
#                   signal_features=None, n_signal_features=None, permute_nonsignal_col=None):
#     """
#     :param fpath: path to X data
#     :param X: data matrix
#     :param seed: random seed
#     :param normalize: boolean; whether or not to normalize columns in data to mean 0 and variance 1
#     :param sample_row_n: number of samples to subset; default keeps all rows
#     :param sample_col_n: number of features to subset; default keeps all columns
#     :param permute_col: boolean; whether or not to permute the columns
#     :param signal_features: list of features to use as signal features
#     :param n_signal_features: number of signal features; required if permute_nonsignal_col is not None
#     :param permute_nonsignal_col: how to permute the nonsignal features; must be one of
#         [None, "block", "indep", "augment"], where None performs no permutation, "block" performs the permutation
#         row-wise, "indep" permutes each nonsignal feature column independently, "augment" augments the signal features
#         with the row-permuted X matrix.
#     :return:
#     """
#     assert permute_nonsignal_col in [None, "block", "indep", "augment"]
#     if X is None:
#         X = pd.read_csv(fpath)
#     if normalize:
#         X = (X - X.mean()) / X.std()
#     if seed is not None:
#         np.random.seed(seed)
#     if permute_col:
#         X = X[np.random.permutation(X.columns)]
#     if sample_row_n is not None:
#         keep_idx = np.random.choice(X.shape[0], sample_row_n, replace=False)
#         X = X.iloc[keep_idx, :]
#     if sample_col_n is not None:
#         if signal_features is None:
#             X = X.sample(n=sample_col_n, replace=False, axis=1)
#         else:
#             rand_features = np.random.choice([col for col in X.columns if col not in signal_features],
#                                              sample_col_n - len(signal_features), replace=False)
#             X = X[signal_features + list(rand_features)]
#     if signal_features is not None:
#         X = X[signal_features + [col for col in X.columns if col not in signal_features]]
#     if permute_nonsignal_col is not None:
#         assert n_signal_features is not None
#         if permute_nonsignal_col == "block":
#             X = np.hstack([X.iloc[:, :n_signal_features].to_numpy(),
#                            X.iloc[np.random.permutation(X.shape[0]), n_signal_features:].to_numpy()])
#             X = pd.DataFrame(X)
#         elif permute_nonsignal_col == "indep":
#             for j in range(n_signal_features, X.shape[1]):
#                 X.iloc[:, j] = np.random.permutation(X.iloc[:, j])
#         elif permute_nonsignal_col == "augment":
#             X = np.hstack([X.iloc[:, :n_signal_features].to_numpy(),
#                            X.iloc[np.random.permutation(X.shape[0]), :].to_numpy()])
#             X = IndexedArray(pd.DataFrame(X).to_numpy(), index=keep_idx)
#             return X
#     return X.to_numpy()

# def sample_real_Y(X, fpath=None, return_support=False):
#     Y = pd.read_csv(fpath)
#     if return_support:
#         return Y.to_numpy(), np.ones(Y.shape[1]), None
#     return Y.to_numpy()

In [20]:
# import pandas as pd
# X = sample_real_X('../data/X_ccle_rnaseq_cleaned_filtered5000.csv')
# y = sample_real_Y(X, '../data/y_ccle_AZD0530.csv').flatten()

In [21]:
# # Only select the first 100 samples for demonstration
# X = X[:,:100]

In [22]:
# y.shape
(200,1)
(200,)

(200,)

In [23]:
n = 200
d = 10
mean = [[0]*5 + [0]*5, [10]*5 + [0]*5]
scale = [[1]*10,[1]*10]
s = 5
X = sample_normal_X_subgroups(n, d, mean, scale)
beta = np.concatenate((np.ones(s), np.zeros(d-s)))
y = np.matmul(X, beta)
split_seed = 0
X_train, X_tune, X_test, y_train, y_tune, y_test = apply_splitting_strategy(X, y, "train-test", split_seed)

In [24]:
# turn y.shape to (200,1)
y = y.reshape(-1,1)

In [25]:
rf_regressor = RandomForestRegressor(n_estimators=100, min_samples_leaf=5, max_features=0.33, random_state=331)
rf_regressor.fit(X_train, y_train)
seed = 0

In [26]:
rf_plus_model = RandomForestPlusRegressor(rf_model=copy.deepcopy(rf_regressor), include_raw=False)
# rf_plus_model = RandomForestPlusRegressor(rf_model=rf_regressor, include_raw=False)
rf_plus_model.fit(X_train, y_train)

score = rf_plus_model.get_mdi_plus_scores(X_test, y_test, lfi=True, lfi_abs = "none", sample_split=None, train_or_test = "test")
score["lfi"]

ValueError: Transformer representation was empty for all trees.

In [ ]:
rf_plus_model = RandomForestPlusRegressor(rf_model=copy.deepcopy(rf_regressor), include_raw=True)
# rf_plus_model = RandomForestPlusRegressor(rf_model=rf_regressor, include_raw=True)
rf_plus_model.fit(X_train, y_train)

score = rf_plus_model.get_mdi_plus_scores(X_test, y_test, lfi=True, lfi_abs = "none", sample_split=None, train_or_test = "test")
score["lfi"]

In [ ]:
score = rf_plus_model.get_mdi_plus_scores(X_test, y_test, lfi=True, lfi_abs = "none", sample_split=None, train_or_test = "test")
score["lfi"]

In [ ]:
# Define the data
n = 200
d = 10
mean = [[0]*5 + [0]*5, [10]*5 + [0]*5]
scale = [[1]*10,[1]*10]
s = 5
X = sample_normal_X_subgroups(n, d, mean, scale)
beta = np.concatenate((np.ones(s), np.zeros(d-s)))
y = np.matmul(X, beta)
split_seed = 0
X_train, X_tune, X_test, y_train, y_tune, y_test = apply_splitting_strategy(X, y, "train-test", split_seed)

# #Define the model and fit
# rf_regressor = RandomForestRegressor(n_estimators=100, min_samples_leaf=5, max_features=0.33, random_state=331)
# rf_regressor.fit(X_train, y_train)
# seed = 0
# # pass in a copy of rf_regressor to avoid modifying the original model
# rf_plus_model = RandomForestPlusRegressor(rf_model=copy.deepcopy(rf_regressor), include_raw=False)
# rf_plus_model = RandomForestPlusRegressor(rf_model=rf_regressor, include_raw=False)
# rf_plus_model.fit(X_train, y_train)

# # initialize the metric results
# metric_results = {}

# y_pred = rf_regressor.predict(X_test)
# metric_results['MSE_before_ablation'] = mean_squared_error(y_test, y_pred)

# # Ablation
# score = rf_plus_model.get_mdi_plus_scores(X_test, y_test, lfi=True, lfi_abs = "outside", sample_split=None)
# local_fi_score = score["lfi"]
# ascending = True # False for MDI
# imp_vals = copy.deepcopy(local_fi_score)
# imp_vals[imp_vals == float("-inf")] = -sys.maxsize - 1
# imp_vals[imp_vals == float("inf")] = sys.maxsize - 1
# seed = 0 #np.random.randint(0, 100000)
# for i in range(X_test.shape[1]):
#     if ascending:
#         ablation_X_test = ablation(X_test, imp_vals, "max", i+1, seed)
#     else:
#         ablation_X_test = ablation(X_test, imp_vals, "min", i+1, seed)
#     metric_results[f'MSE_after_ablation_{i+1}'] = mean_squared_error(y_test, rf_regressor.predict(ablation_X_test))

In [ ]:
#Define the model and fit
rf_regressor = RandomForestRegressor(n_estimators=100, min_samples_leaf=5, max_features=0.33, random_state=331)
rf_regressor.fit(copy.deepcopy(X_train), copy.deepcopy(y_train))
seed = 0
# pass in a copy of rf_regressor to avoid modifying the original model
#rf_plus_model = RandomForestPlusRegressor(rf_model=copy.deepcopy(rf_regressor), include_raw=False)
rf_plus_model = RandomForestPlusRegressor(rf_model=rf_regressor, include_raw=False)
rf_plus_model.fit(copy.deepcopy(X_train), copy.deepcopy(y_train))

In [ ]:
# #plot metric_results
# fig, ax = plt.subplots()
# ax.plot(list(metric_results.keys()), list(metric_results.values()))
# ax.set_xlabel('Number of features ablated')
# ax.set_ylabel('MSE')
# ax.set_title('MSE after ablation')
# plt.show()

In [ ]:
metric_results = {}
y_pred = rf_regressor.predict(X_test)
metric_results['MSE_before_ablation'] = mean_squared_error(y_test, y_pred)
local_fi_score = tree_shap_local(X_test, y_test, rf_regressor)
ascending = True # False for MDI
imp_vals = copy.deepcopy(local_fi_score)
imp_vals[imp_vals == float("-inf")] = -sys.maxsize - 1
imp_vals[imp_vals == float("inf")] = sys.maxsize - 1
seed = np.random.randint(0, 100000)
for i in range(X_test.shape[1]):
    if ascending:
        ablation_X_test = ablation(X_test, imp_vals, "max", i+1, seed)
    else:
        ablation_X_test = ablation(X_test, imp_vals, "min", i+1, seed)
    metric_results[f'MSE_after_ablation_{i+1}'] = mean_squared_error(y_test, rf_regressor.predict(ablation_X_test))

In [ ]:
# plot metric_results
fig, ax = plt.subplots()
ax.plot(list(metric_results.keys()), list(metric_results.values()))
ax.set_xlabel('Number of features ablated')
ax.set_ylabel('MSE')
ax.set_title('MSE after ablation')
plt.show()

In [ ]:
# n = 200
# d = 10
# mean = [[100,70,50,30,10]+[0]*5, [100,70,50,30,10]+[0]*5]
# scale = [[1]*10,[1]*10]
# s = 5
# X = sample_normal_X_subgroups(n, d, mean, scale)
# beta = np.concatenate((np.ones(s), np.zeros(d-s)))
# y = np.matmul(X, beta)
# split_seed = 0
# X_train, X_tune, X_test, y_train, y_tune, y_test = apply_splitting_strategy(X, y, "train-test", split_seed)

# #Define the model and fit
# rf_regressor = RandomForestRegressor(n_estimators=100, min_samples_leaf=5, max_features=0.33, random_state=331)
# rf_regressor.fit(X_train, y_train)
# seed = 0

# metric_results = {}
# y_pred = rf_regressor.predict(X_test)
# metric_results['MSE_before_ablation'] = mean_squared_error(y_test, y_pred)
# local_fi_score = tree_shap_local(X_test, y_test, rf_regressor)
# ascending = True # False for MDI
# imp_vals = copy.deepcopy(local_fi_score)
# imp_vals[imp_vals == float("-inf")] = -sys.maxsize - 1
# imp_vals[imp_vals == float("inf")] = sys.maxsize - 1
# seed = np.random.randint(0, 100000)
# for i in range(X_test.shape[1]):
#     if ascending:
#         ablation_X_test = ablation(X_test, imp_vals, "max", i+1, seed)
#     else:
#         ablation_X_test = ablation(X_test, imp_vals, "min", i+1, seed)
#     metric_results[f'MSE_after_ablation_{i+1}'] = mean_squared_error(y_test, rf_regressor.predict(ablation_X_test))

In [ ]:
n = 200
d = 10
mean = [[100,70,50,30,10]+[0]*5, [100,70,50,30,10]+[0]*5]
scale = [[1]*10,[1]*10]
s = 5
X = sample_normal_X_subgroups(n, d, mean, scale)
beta = np.concatenate((np.ones(s), np.zeros(d-s)))
y = np.matmul(X, beta)
split_seed = 0
X_train, X_tune, X_test, y_train, y_tune, y_test = apply_splitting_strategy(X, y, "train-test", split_seed)

#Define the model and fit
rf_regressor = RandomForestRegressor(n_estimators=100, min_samples_leaf=5, max_features=0.33, random_state=331)
rf_regressor.fit(X_train, y_train)
seed = 0

metric_results = {}
y_pred = rf_regressor.predict(X_test)
metric_results['MSE_before_ablation'] = mean_squared_error(y_test, y_pred)
local_fi_score = tree_shap_local(X_test, y_test, rf_regressor)
ascending = True # False for MDI
imp_vals = copy.deepcopy(local_fi_score)
imp_vals[imp_vals == float("-inf")] = -sys.maxsize - 1
imp_vals[imp_vals == float("inf")] = sys.maxsize - 1
seed = np.random.randint(0, 100000)
for i in range(X_test.shape[1]):
    if ascending:
        ablation_X_test = ablation(X_test, imp_vals, "max", i+1, seed)
    else:
        ablation_X_test = ablation(X_test, imp_vals, "min", i+1, seed)
    metric_results[f'MSE_after_ablation_{i+1}'] = mean_squared_error(y_test, rf_regressor.predict(ablation_X_test))

In [ ]:
# plot metric_results
fig, ax = plt.subplots()
ax.plot(list(metric_results.keys()), list(metric_results.values()))
ax.set_xlabel('Number of features ablated')
ax.set_ylabel('MSE')
ax.set_title('MSE after ablation')
plt.show()

In [ ]:
n = 200
d = 10
mean = [[100,70,50,30,10]+[0]*5, [10,7,5,3,1]+[0]*5]
scale = [[1]*10,[1]*10]
s = 5
X = sample_normal_X_subgroups(n, d, mean, scale)
beta = np.concatenate((np.ones(s), np.zeros(d-s)))
y = np.matmul(X, beta)
split_seed = 0
X_train, X_tune, X_test, y_train, y_tune, y_test = apply_splitting_strategy(X, y, "train-test", split_seed)

#Define the model and fit
rf_regressor = RandomForestRegressor(n_estimators=100, min_samples_leaf=5, max_features=0.33, random_state=331)
rf_regressor.fit(X_train, y_train)
seed = 0

metric_results = {}
y_pred = rf_regressor.predict(X_test)
metric_results['MSE_before_ablation'] = mean_squared_error(y_test, y_pred)
local_fi_score = tree_shap_local(X_test, y_test, rf_regressor)
ascending = True # False for MDI
imp_vals = copy.deepcopy(local_fi_score)
imp_vals[imp_vals == float("-inf")] = -sys.maxsize - 1
imp_vals[imp_vals == float("inf")] = sys.maxsize - 1
seed = np.random.randint(0, 100000)
for i in range(X_test.shape[1]):
    if ascending:
        ablation_X_test = ablation(X_test, imp_vals, "max", i+1, seed)
    else:
        ablation_X_test = ablation(X_test, imp_vals, "min", i+1, seed)
    metric_results[f'MSE_after_ablation_{i+1}'] = mean_squared_error(y_test, rf_regressor.predict(ablation_X_test))

In [ ]:
# plot metric_results
fig, ax = plt.subplots()
ax.plot(list(metric_results.keys()), list(metric_results.values()))
ax.set_xlabel('Number of features ablated')
ax.set_ylabel('MSE')
ax.set_title('MSE after ablation')
plt.show()

In [ ]:
coolwarm_camp = sns.color_palette("coolwarm", as_cmap=True)
rf_regressor.fit(X_train, y_train)
shap_val = tree_shap_local(X_test, y_test, rf_regressor)
shap_val.columns = [f'{i}' for i in range(X.shape[1])]
sns.heatmap(shap_val, cmap=coolwarm_camp, xticklabels="auto", yticklabels="auto")
plt.title("SHAP Values")
plt.xlabel("Feature Number")
plt.ylabel("Feature Importance Score - higher absolute better")